In [1]:
!rm -r /content/sample_data/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/colab_data/diplom/'
path_data = path + 'data/'

In [ ]:
# Раскомментировать, если данные упакованы в zip архив

# !unzip $path_data'train.zip' -d $path_data
# !unzip $path_data'test_kaggle.zip' -d $path_data
# !unzip $path_data'skillbox-computer-vision-project.zip' -d $path_data''

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os

from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [5]:
# Добавляем классы в словарь
classes = {0: 'anger',\
           1: 'contempt',\
           2: 'disgust',\
           3: 'fear',\
           4: 'happy',\
           5: 'neutral',\
           6: 'sad',\
           7: 'surprise',
           8: 'uncertain'}

In [7]:
#Константы

BATCH_SIZE = 128
IMG_SIZE = 48
EPOCHS = 130
STEPS_PER_EPOCH = 60

In [8]:
#Модель

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

# model.add(Dense(7, activation='softmax'))
# model.load_weights(path + 'model.h5')
# model.pop()

model.add(Dense(9, activation='softmax'))

model.load_weights(path + 'mdl_wts.hdf5')

#На случай, если необходимо заморозить слои
# for l in model.layers[:-1]:
  # l.trainable = False

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [10]:
# Создаём поток из фотографий
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        path_data+'train',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        color_mode="grayscale",
        class_mode='categorical')

Found 50046 images belonging to 9 classes.


In [11]:
# Обучаем и сохраняем веса модели
mcp = ModelCheckpoint(path + 'mdl_wts.hdf5',\
                      save_best_only=True,\
                      monitor='accuracy')
opt = Adam(lr=0.0001, decay=1e-6)


model.compile(loss='categorical_crossentropy',\
              optimizer=opt,\
              metrics=['accuracy'])
model.fit(train_generator,\
          epochs=EPOCHS,\
          steps_per_epoch=STEPS_PER_EPOCH,\
          callbacks=[mcp])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/130
60/60 [==============================] - 1572s 26s/step - loss: 0.6040 - accuracy: 0.7858
Epoch 2/130
60/60 [==============================] - 1343s 22s/step - loss: 0.6068 - accuracy: 0.7901
Epoch 3/130
60/60 [==============================] - 1129s 19s/step - loss: 0.5998 - accuracy: 0.7865
Epoch 4/130
60/60 [==============================] - 968s 16s/step - loss: 0.5885 - accuracy: 0.7910
Epoch 5/130
60/60 [==============================] - 806s 13s/step - loss: 0.5894 - accuracy: 0.7894
Epoch 6/130
60/60 [==============================] - 692s 12s/step - loss: 0.6208 - accuracy: 0.7802
Epoch 7/130
60/60 [==============================] - 588s 10s/step - loss: 0.5883 - accuracy: 0.7927
Epoch 8/130
60/60 [==============================] - 514s 9s/step - loss: 0.6060 - accuracy: 0.7846
Epoch 9/130
60/60 [==============================] - 438s 7s/step - loss: 0.6001 - accuracy: 0.7836
Epoch 10/130
60/60 [==============================] - 378s 6s/step - loss: 0.5995 - accura

In [12]:
# Оптимизируем модель с помощью TensorRT
model.load_weights(path + 'mdl_wts.hdf5')

model.save(path+'saved_model')

converter = trt.TrtGraphConverterV2(input_saved_model_dir=path+'saved_model')
converter.convert()
converter.save(path+'saved_model_trt')

model_trt = tf.keras.models.load_model(path+'saved_model_trt')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/colab_data/diplom/saved_model/assets
INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: /content/drive/MyDrive/colab_data/diplom/saved_model_trt/assets


In [14]:
TEST_BATCH_SIZE = 100

out = []

imgs = []
files = [] 

# Прогоняем все файлы для теста

batch_num = 0

for filepath in os.listdir(path_data+'test_kaggle/'):
  files.append(filepath)
  imgs.append((np.array(load_img(path_data+'test_kaggle/{0}'.format(filepath),\
                       color_mode='grayscale',\
                       target_size=(IMG_SIZE,IMG_SIZE)), dtype=np.float32) / 255.)[..., None])
  
  if len(imgs) == TEST_BATCH_SIZE:
    imgs = np.array(imgs, dtype=np.float32)

    predictions = model_trt(imgs)
    predictions = [classes[predict] for predict in np.argmax(predictions, axis=1)]

    out += list(zip(files, predictions))

    imgs = []
    files = []

    batch_num += 1
    print(batch_num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [15]:
# Записываем результат в csv файл

out = pd.DataFrame(out, columns=['image_path', 'emotion'])
out['sort_val'] = out.image_path.apply(lambda x: int(x.replace('.jpg', '')))
out = out.sort_values('sort_val')
out = out.drop('sort_val', 1)

compression_opts = dict(method='zip', archive_name='out.csv') 

out.to_csv(path+'out.zip', index=False, compression=compression_opts)  